In [2]:
import os 
from ctypes import * # read the functions from the cdb
from sofistik_daten import * ##importer les classes sofistik
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from shapely.geometry import *


# Strut and Tie generation using Sofistik

## I. Connection to the CDB

Use Sofistik library : https://docs.sofistik.com/2024/en/cdb_interfaces/python/examples/python_example1.html

In [3]:
# Set DLL dir path
# os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024\interfaces\64bit")
os.add_dll_directory(r"C:\Program Files\SOFiSTiK\2024\SOFiSTiK 2024")

# Get the DLL functions
myDLL = cdll.LoadLibrary("sof_cdb_w-2024.dll")
py_sof_cdb_get = cdll.LoadLibrary("sof_cdb_w-2024.dll").sof_cdb_get
py_sof_cdb_get.restype = c_int
py_sof_cdb_kenq = cdll.LoadLibrary("sof_cdb_w-2024.dll").sof_cdb_kenq_ex

def open_cdb(fileName):

    # Connect to CDB
    Index = c_int()
    cdbIndex = 99

    # important: Unicode call!
    Index.value = myDLL.sof_cdb_init(fileName.encode('utf8'), cdbIndex)

    # get the CDB status
    cdbStat = c_int()
    cdbStat.value = myDLL.sof_cdb_status(Index.value)

    # Print the Status of the CDB
    print ("CDB opened successfully, CDB Status =", cdbStat.value)

    return cdbStat, Index

def close_cdb(cdbStat, Index):

    myDLL = cdll.LoadLibrary("sof_cdb_w-2024.dll")
    # Close the CDB, 0 - will close all the files
    myDLL.sof_cdb_close(0)
    
    # Print again the status of the CDB, if status = 0 -> CDB Closed successfully
    cdbStat.value = myDLL.sof_cdb_status(Index.value)
    if cdbStat.value == 0:
        print ("CDB closed successfully, CDB Status = 0")

fileName=r"sofistik_cdb\b.cdb"

cdbStat, Index=open_cdb(fileName) 
close_cdb(cdbStat, Index)
    

CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0


<span style="background-color: darkred; font-size:24px;">cdbStat</span> : C'est une variable qui sert à vérifier le statut de chaque opération sur la base de données. Elle est utile pour s'assurer que chaque étape (ouverture, fermeture, lecture) s'est déroulée correctement, et elle permet de diagnostiquer les erreurs éventuelles.

<span style="background-color: darkred;font-size:24px;">Index</span> : C'est un identifiant unique qui permet de manipuler la base de données CDB tout au long du programme. Il est utilisé pour garder une trace de la connexion à la CDB et est nécessaire pour les opérations de lecture et d'écriture.

## II. Accéder aux forces dans les éléments du maillage

In [18]:
# Function to read nodal forces using CN_DISP structure and store in a list
def quad_forces(fileName):
    cdbStat,Index=open_cdb(fileName)
    ie = c_int(0)
    RecLen = c_int(sizeof(CQUAD_FOR))  # Size of the displacement and forces structure
    forces = CQUAD_FOR()  # Instance of the displacement and forces structure
    forces_data = []  # List to store forces for each 

    while ie.value < 2:
        ie.value = py_sof_cdb_get(Index, 210, 1, byref(forces), byref(RecLen), 1)

        if ie.value == 0:  # No error, data was read
            forces_data.append({
                "quad_number": forces.m_nr,   # Quad number
                "mxx": forces.m_mxx,          # Bending moment mxx
                "myy": forces.m_myy,          # Bending moment myy
                "mxy": forces.m_mxy,          # Bending moment mxy
                "vx": forces.m_vx,            # Shear force vx
                "vy": forces.m_vy,            # Shear force vy
                "nx": forces.m_nx,            # Membrane force nx
                "ny": forces.m_ny,            # Membrane force ny
                "nxy": forces.m_nxy           # Membrane force nxy
            })
        
        # Always reset RecLen before the next sof_cdb_get call
        RecLen = c_int(sizeof(CQUAD_FOR))
    close_cdb(cdbStat, Index)
    return forces_data  # Return the list with all the forces data

def forces_to_pd(forces_list):
    return pd.DataFrame(forces_list)

def membran_forces(df_forces):
    df_membran_forces=df_forces[['quad_number','nx','ny','nxy']]
    df_membran_forces = df_membran_forces.iloc[2:].reset_index(drop=True) ##Supprimer les deux premières lignes qui ne donnent pas la bonne information
    return df_membran_forces

# Example of usage
fileName = r"sofistik_cdb\b.cdb"
df_membran_forces = membran_forces(forces_to_pd(quad_forces(fileName)))

CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0


In [10]:
df_membran_forces

,quad_number,nx,ny,nxy
0,20001,-5.888048,-19.540747,-22.777464
1,20002,-2.871248,-31.321043,-23.620758
2,20003,-1.245961,-32.178619,-17.236378
3,20004,-3.722728,-22.488720,-19.172930
4,20005,33.643772,0.065014,1.881605
...,...,...,...,...
285,20286,-3.715370,-7.800370,7.062850
286,20287,-6.273705,-5.901103,7.507652
287,20288,-2.582406,-26.801802,19.760466
288,20289,-2.137352,-23.699650,15.294311


## III. Maillage

On désire visualiser le maillage qu'on obtient sur Sofistik, puis visualiser les forces dans chaque maille.

In [21]:
# Fonction pour extraire les coordonnées des nœuds depuis la base de données CDB
def extract_node_coords(Index):
    ie = c_int(0)
    RecLen = c_int(sizeof(CNODE))  # Taille de la structure CNODE
    node = CNODE()  # Instance de la structure CNODE
    node_coords = {}  # Dictionnaire pour stocker les coordonnées des nœuds

    while ie.value < 2:
        ie.value = py_sof_cdb_get(Index, 20, 0, byref(node), byref(RecLen), 1)  # Lire les nœuds depuis CDB
        if ie.value == 0:  # Pas d'erreur, nœud lu correctement
            node_coords[node.m_nr] = (node.m_xyz[0], node.m_xyz[1], node.m_xyz[2])  # Stocker les coordonnées X, Y, Z du nœud
        RecLen = c_int(sizeof(CNODE))  # Réinitialiser la taille de l'enregistrement
    return node_coords  # Retourner les coordonnées des nœuds

# Fonction pour extraire les éléments quadrilatéraux (avec les nœuds associés)
def extract_quad_elements(Index):
    ie = c_int(0)
    RecLen = c_int(sizeof(CQUAD))  # Taille de la structure CQUAD
    quad_element = CQUAD()  # Instance de la structure CQUAD
    elements = []  # Liste pour stocker les éléments quadrilatéraux
    element_numbers=[]
    while ie.value < 2:
        ie.value = py_sof_cdb_get(Index, 200, 0, byref(quad_element), byref(RecLen), 1)  # Lire les éléments quadrilatéraux
        if ie.value == 0:  # Pas d'erreur, élément lu correctement
            elements.append(quad_element.m_node[:])  # Ajouter les 4 nœuds de l'élément dans la liste
            element_numbers.append(quad_element.m_nr)
        RecLen = c_int(sizeof(CQUAD))  # Réinitialiser la taille de l'enregistrement

    return elements,element_numbers  # Retourner les coordonnées et le numéro de chaque élément

# Fonction pour créer et afficher le maillage quadrilatéral 2D interactif avec Plotly en fonction des axes spécifiés
def plot_quad_mesh_from_cdb(file_path, view_axis='xy'):
    # Ouverture de la base de données CDB
    cdbStat, Index = open_cdb(file_path)

    # Extraction des coordonnées des nœuds
    node_coords = extract_node_coords(Index)

    # Extraction des éléments quadrilatéraux
    elements,_ = extract_quad_elements(Index)

    # Fermeture de la base de données CDB
    close_cdb(cdbStat, Index)

    # Création d'une figure interactive 2D avec Plotly
    fig = go.Figure()

    # Sélection des axes pour la visualisation 2D
    axis_map = {
        'xy': (0, 1),  # Utiliser les axes X et Y
        'xz': (0, 2),  # Utiliser les axes X et Z
        'yz': (1, 2)   # Utiliser les axes Y et Z
    }
    ax1, ax2 = axis_map.get(view_axis, (0, 1))  # Par défaut, afficher selon XY

    # Tracer les éléments quadrilatéraux
    for element in elements:
        # Récupérer les coordonnées des nœuds de l'élément
        quad_coords = np.array([node_coords[node] for node in element])
        quad_coords = np.vstack([quad_coords, quad_coords[0]])  # Fermer le quadrilatère

        # Sélectionner les axes en fonction de la vue choisie
        fig.add_trace(go.Scatter(
            x=quad_coords[:, ax1], y=quad_coords[:, ax2],
            mode='lines',
            line=dict(color='blue', width=2),
            showlegend=False
        ))

    # Configurer le layout selon les axes choisis
    axis_labels = {
        'xy': ('X', 'Y'),
        'xz': ('X', 'Z'),
        'yz': ('Y', 'Z')
    }
    x_label, y_label = axis_labels.get(view_axis, ('X', 'Y'))

    fig.update_layout(
        title=f"Quadrilateral Mesh in 2D (View: {view_axis})",
        xaxis_title=x_label,
        yaxis_title=y_label,
        autosize=True,
        showlegend=False,
        xaxis=dict(scaleanchor="y", scaleratio=1),
        yaxis=dict(scaleanchor="x", scaleratio=1)
    )

    # Affichage interactif
    fig.show()

# Exemple d'utilisation
file_path = r"sofistik_cdb\b.cdb"

# Choisir l'axe de vue (par exemple, 'xy', 'xz', 'yz')
plot_quad_mesh_from_cdb(file_path, view_axis='xy')


CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0


In [24]:
# Fonction pour calculer le centre d'un quadrilatère
def calculate_quad_centers(node_coords, elements):
    centers = []
    for element in elements:
        # Extraire les coordonnées des nœuds de l'élément
        quad_coords = np.array([node_coords[node] for node in element])
        # Calculer le centre du quadrilatère en prenant la moyenne des coordonnées
        center = np.mean(quad_coords, axis=0)
        centers.append(center)
    return centers

import numpy as np
import plotly.graph_objects as go

# Fonction pour afficher le maillage de la structure et les forces selon la direction spécifiée
def plot_quad_mesh_for(file_path, df_membran_forces, force_column='nx', view_axis='xy'):
    # Ouverture de la base de données CDB
    cdbStat, Index = open_cdb(file_path)

    # Extraction des coordonnées des nœuds
    node_coords = extract_node_coords(Index)

    # Extraction des éléments quadrilatéraux
    elements,_ = extract_quad_elements(Index)

    # Fermeture de la base de données CDB
    close_cdb(cdbStat, Index)

    # Calcul des centres des éléments quadrilatéraux
    centers = calculate_quad_centers(node_coords, elements)

    # Création d'une figure plotly
    fig = go.Figure()

    # Définir les axes en fonction de la vue choisie
    axis_map = {
        'xy': (0, 1),  # Utiliser les axes X et Y
        'xz': (0, 2),  # Utiliser les axes X et Z
        'yz': (1, 2)   # Utiliser les axes Y et Z
    }
    ax1, ax2 = axis_map.get(view_axis, (0, 1))  # Par défaut, afficher selon XY

    # Tracer le maillage sans valeurs
    for element in elements:
        quad_coords = np.array([node_coords[node] for node in element])
        quad_coords = np.vstack([quad_coords, quad_coords[0]])  # Fermer le quadrilatère
        x_coords = quad_coords[:, ax1]
        y_coords = quad_coords[:, ax2]

        # Ajout du quadrilatère au graphique
        fig.add_trace(go.Scatter(
            x=x_coords, y=y_coords, mode='lines',
            line=dict(color='black', width=1),
            showlegend=False
        ))

    # Affichage des valeurs des forces au centre de chaque quadrilatère
    for i, center in enumerate(centers):
        force_value = df_membran_forces.loc[i, force_column]

        # Déterminer la couleur : rouge pour négatif, bleu pour positif
        color = 'red' if force_value < 0 else 'blue'

        # Ajouter la valeur de la force comme annotation
        fig.add_trace(go.Scatter(
            x=[center[ax1]], y=[center[ax2]],
            text=[f'{force_value:.2f}'],
            mode='text',
            textfont=dict(color=color, size=12)
        ))

    # Configuration du layout pour interactivité
    axis_labels = {
        'xy': ('X', 'Y'),
        'xz': ('X', 'Z'),
        'yz': ('Y', 'Z')
    }
    x_label, y_label = axis_labels.get(view_axis, ('X', 'Y'))

    fig.update_layout(
        title=f"{force_column} Values on Quadrilateral Mesh (View: {view_axis}, Interactive)",
        xaxis_title=x_label,
        yaxis_title=y_label,
        xaxis=dict(scaleanchor="y", scaleratio=1),  # Forcing equal scaling
        yaxis=dict(scaleanchor="x", scaleratio=1),
        showlegend=False,
        autosize=True,
    )

    # Affichage interactif
    fig.show()

In [26]:
# Exemple d'utilisation
file_path = r"sofistik_cdb\b.cdb"

# DataFrame df_membran_forces doit être défini à l'avance
# Afficher les forces normales selon x (nx) dans le plan xy
plot_quad_mesh_for(file_path, df_membran_forces, force_column='nx', view_axis='xy')

# # Afficher les forces normales selon y (ny) dans le plan xz
# plot_quad_mesh_for(file_path, df_membran_forces, force_column='ny', view_axis='xz')

CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0


## IV. Ajouter des tirants

In [33]:
def extremites(Index):
    syst = CSYST()
    RecLen = c_int(sizeof(CSYST))
    ie = c_int(0)
    x=[]
    while ie.value < 2:
        # Lire les données SYST (10/00)
        ie.value = py_sof_cdb_get(Index, 10,00, byref(syst), byref(RecLen), 1)
        if ie.value ==0:
            # Extraction des valeurs de m_box (2x3)
            for i in range(3):
                for j in range(2):
                    x.append(syst.m_box[i][j])
            return x

In [36]:
def add_tirants(file_path, dir='y', n=5, m=5):
    """
    Génère la liste des tirants sans afficher la figure.
    
    :param file_path: Chemin vers le fichier CDB
    :param view_axis: Vue sélectionnée ('xy', 'xz', 'yz')
    :param n: Nombre de tirants à ajouter en X
    :param m: Nombre de tirants à ajouter en Y ou Z
    :return: Liste des tirants ajoutés
    """
    # Ouverture de la base de données CDB
    cdbStat, Index = open_cdb(file_path)

    x=extremites(Index)

    # Fermeture de la base de données CDB
    close_cdb(cdbStat, Index)

    # Initialisation
    tirants = []
    x_min, y_min, z_min = x[:3]
    x_max, y_max, z_max = x[3:]

    #Ajout de la marge
    margin = 0.1
    x_min,y_min,z_min = x_min+margin,y_min+margin,z_min+margin
    x_max,y_max,z_max=x_max-margin,y_max-margin,z_max-margin

    # Ajout de tirants en fonction des paramètres n et m
    tirant_spacing_x = (x_max - x_min) / n
    tirant_spacing_y = (y_max - y_min) / m
    tirant_spacing_z = (z_max - z_min) / m

    
    if dir=='y':
        # Tirants en X    
        for i in range(m + 1):
            y = y_min + i * tirant_spacing_y
            tirants.append(((x_min, y), (x_max, y)))
        # Tirants en Y
        for i in range(n + 1):
            x = x_min + i * tirant_spacing_x
            tirants.append(((x, y_min), (x, y_max)))        
    else:
        # Tirants en X
        for i in range(m + 1):
            z = z_min + i * tirant_spacing_z
            tirants.append(((x_min, z), (x_max, z)))
        # Tirants en Z
        for i in range(n + 1):
            x = x_min + i * tirant_spacing_x
            tirants.append(((x, z_min), (x, z_max)))

    # Retourner la liste des tirants
    return tirants

def add_fig_tirants(fig, file_path, dir='y', n=5, m=5):
    """
    Ajoute les tirants (lignes vertes) à la figure fig en fonction des extrémités extraites de la base de données CDB.
    Utilise la fonction add_tirants pour générer les tirants avec une marge de 0.1 m aux bords.
    
    :param fig: Figure Plotly à laquelle les lignes seront ajoutées
    :param file_path: Chemin vers le fichier CDB
    :param dir: Direction des tirants ('y' pour le plan XY, 'z' pour le plan XZ)
    :param n: Nombre de lignes dans la direction X
    :param m: Nombre de lignes dans la direction Y ou Z
    :return: Figure mise à jour et liste des tirants avec coordonnées des extrémités
    """
    # Générer les tirants en utilisant la fonction add_tirants
    tirants = add_tirants(file_path, dir,n,m)

    # Ajouter les tirants à la figure
    for tirant in tirants:
        # Extraire les coordonnées des extrémités des tirants
        (x1, y1), (x2, y2) = tirant
        # Ajouter la ligne verte à la figure
        fig.add_trace(go.Scatter(
            x=[x1, x2], y=[y1, y2], mode='lines',
            line=dict(color='green', width=2, dash='dash'),
            showlegend=False
        ))

    # Retourner la figure mise à jour et la liste des tirants
    return fig, tirants

def plot_structure_with_tirants(file_path, view_axis='xy', n=5, m=5):
    """
    Trace la structure avec les lignes vertes (tirants) et retourne la figure mise à jour avec les tirants.
    
    :param file_path: Chemin vers le fichier CDB
    :param view_axis: Vue sélectionnée ('xy', 'xz', 'yz')
    :param n: Nombre de tirants à ajouter dans la direction X
    :param m: Nombre de tirants à ajouter dans la direction Y ou Z
    :return: Figure Plotly mise à jour avec les tirants et la liste des tirants
    """
    # Ouverture de la base de données CDB
    cdbStat, Index = open_cdb(file_path)

    # Extraction des coordonnées des nœuds
    node_coords = extract_node_coords(Index)

    # Extraction des éléments quadrilatéraux
    elements,_ = extract_quad_elements(Index)

    # Fermeture de la base de données CDB
    close_cdb(cdbStat, Index)

    # Création d'une figure plotly
    fig = go.Figure()

    # Définir les axes en fonction de la vue choisie
    axis_map = {
        'xy': (0, 1),  # Utiliser les axes X et Y
        'xz': (0, 2),  # Utiliser les axes X et Z
        'yz': (1, 2)   # Utiliser les axes Y et Z
    }
    ax1, ax2 = axis_map.get(view_axis, (0, 1))  # Par défaut, afficher selon XY

    # Tracer le maillage sans valeurs
    for element in elements:
        quad_coords = np.array([node_coords[node] for node in element])
        quad_coords = np.vstack([quad_coords, quad_coords[0]])  # Fermer le quadrilatère
        x_coords = quad_coords[:, ax1]
        y_coords = quad_coords[:, ax2]

        # Ajout du quadrilatère au graphique
        fig.add_trace(go.Scatter(
            x=x_coords, y=y_coords, mode='lines',
            line=dict(color='black', width=1),
            showlegend=False
        ))

    # Utiliser la fonction add_fig_tirants pour ajouter les tirants à la figure
    fig, tirants = add_fig_tirants(fig, file_path, dir=view_axis[1], n=n, m=m)

    # Configuration du layout pour interactivité
    axis_labels = {
        'xy': ('X', 'Y'),
        'xz': ('X', 'Z'),
        'yz': ('Y', 'Z')
    }
    x_label, y_label = axis_labels.get(view_axis, ('X', 'Y'))

    fig.update_layout(
        title=f"Structure with Tirants (View: {view_axis})",
        xaxis_title=x_label,
        yaxis_title=y_label,
        xaxis=dict(scaleanchor="y", scaleratio=1),  # Forcer l'échelle égale
        yaxis=dict(scaleanchor="x", scaleratio=1),
        showlegend=False,
        autosize=True,
    )

    # Retourner la figure mise à jour et la liste des tirants
    return fig, tirants

# Exemple d'utilisation
file_path = r"sofistik_cdb\b.cdb"

# Générer la figure avec la structure et les tirants dans le plan XY
fig, tirants = plot_structure_with_tirants(file_path, view_axis='xy', n=5, m=5)

# Afficher la figure (si vous souhaitez afficher après)
fig.show()

# # Afficher la liste des tirants (coordonnées des extrémités)
# print("Tirants avec coordonnées des extrémités :")
# for tirant in tirants:
#     print(tirant)


CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0
CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0


Tirants avec coordonnées des extrémités :
((0.1, 0.1), (4.9, 0.1))
((0.1, 0.45999999999999996), (4.9, 0.45999999999999996))
((0.1, 0.82), (4.9, 0.82))
((0.1, 1.1800000000000002), (4.9, 1.1800000000000002))
((0.1, 1.54), (4.9, 1.54))
((0.1, 1.9), (4.9, 1.9))
((0.1, 0.1), (0.1, 1.9))
((1.0600000000000003, 0.1), (1.0600000000000003, 1.9))
((2.0200000000000005, 0.1), (2.0200000000000005, 1.9))
((2.980000000000001, 0.1), (2.980000000000001, 1.9))
((3.940000000000001, 0.1), (3.940000000000001, 1.9))
((4.9, 0.1), (4.9, 1.9))


In [38]:
# Fonction pour vérifier si un tirant traverse un élément du maillage
def find_elements_crossed_by_tirant_optimized(tirant, node_coords, elements, tol=0.01): 
    """
    Retourne les éléments du maillage traversés par un tirant, avec optimisation par filtrage spatial.
    
    :param tirant: Tuple de deux points (x1, y1), (x2, y2) représentant un tirant
    :param node_coords: Dictionnaire des coordonnées des nœuds
    :param elements: Liste des éléments du maillage, chaque élément est une liste de nœuds
    :param tol: Tolérance ajoutée autour du tirant pour la boîte de filtrage spatiale
    :return: Liste des éléments (quadrilatères) traversés par le tirant
    """
    tirant_line = LineString([tirant[0], tirant[1]])
    min_x, min_y, max_x, max_y = tirant_line.bounds
    bounding_box = box(min_x - tol, min_y - tol, max_x + tol, max_y + tol)

    crossed_elements = []

    for element in elements:
        quad_coords = np.array([node_coords[node] for node in element])
        quad_polygon = Polygon(quad_coords)

        if bounding_box.intersects(quad_polygon):
            if tirant_line.intersects(quad_polygon):
                crossed_elements.append(tuple(element))  # Ajout de l'élément traversé

    return crossed_elements

# Fonction pour tracer la structure, hachurer les éléments traversés, et afficher les tirants

# Fonction pour tracer la structure, hachurer les éléments traversés, et afficher les tirants
def plot_structure_with_hachured_elements(file_path, crossed_elements, tirants, view_axis='xy'):
    """
    Trace la structure, hachure les éléments traversés en bleu, et affiche les tirants.
    
    :param file_path: Chemin vers le fichier CDB
    :param crossed_elements: Liste des éléments traversés
    :param tirants: Liste des tirants (lignes vertes)
    :param view_axis: Plan de visualisation ('xy', 'xz', 'yz')
    """
    # Ouverture de la base de données CDB
    cdbStat, Index = open_cdb(file_path)

    # Extraction des coordonnées des nœuds
    node_coords = extract_node_coords(Index)

    # Extraction des éléments quadrilatéraux
    elements,_ = extract_quad_elements(Index)

    # Fermeture de la base de données CDB
    close_cdb(cdbStat, Index)

    # Création d'une figure plotly
    fig = go.Figure()

    # Définir les axes en fonction de la vue choisie
    axis_map = {'xy': (0, 1), 'xz': (0, 2), 'yz': (1, 2)}
    ax1, ax2 = axis_map.get(view_axis, (0, 1))

    crossed_elements_set = set(crossed_elements)  # Utiliser un set pour accélérer la recherche

    # Tracer le maillage avec arêtes noires et hachurer les éléments traversés
    for element in elements:
        quad_coords = np.array([node_coords[node] for node in element])
        quad_coords = np.vstack([quad_coords, quad_coords[0]])  # Fermer le quadrilatère
        x_coords = quad_coords[:, ax1]
        y_coords = quad_coords[:, ax2]

        # Tracer les arêtes en noir
        fig.add_trace(go.Scatter(
            x=x_coords, y=y_coords, mode='lines',
            line=dict(color='black', width=2),
            showlegend=False
        ))

        # Si l'élément est traversé, le hachurer en bleu
        if tuple(element) in crossed_elements_set:
            # Remplir l'élément avec une couleur translucide bleue
            fig.add_trace(go.Scatter(
                x=x_coords, y=y_coords, mode='lines', fill='toself',
                fillcolor='rgba(0, 0, 255, 0.2)',  # Bleu translucide
                line=dict(color='black', width=2),
                showlegend=False
            ))

    # Ajouter les tirants verts à la figure
    for tirant in tirants:
        tirant_coords = np.array(tirant)
        x_tirant = tirant_coords[:, ax1]
        y_tirant = tirant_coords[:, ax2]

        fig.add_trace(go.Scatter(
            x=x_tirant, y=y_tirant, mode='lines',
            line=dict(color='green', width=2, dash='dash'),
            showlegend=False
        ))

    # Configuration du layout pour interactivité
    fig.update_layout(
        title="Structure with Hachured Elements and Tirants",
        xaxis_title='X',
        yaxis_title='Y',
        xaxis=dict(scaleanchor="y", scaleratio=1),
        yaxis=dict(scaleanchor="x", scaleratio=1),
        showlegend=False,
        autosize=True,
    )

    # Affichage interactif
    fig.show()

# Exemple d'utilisation
file_path = r"sofistik_cdb\b.cdb"
fig,tirants = plot_structure_with_tirants(file_path, view_axis='xy', n=5, m=5)

cdbStat, Index = open_cdb(file_path)
node_coords = extract_node_coords(Index)
elements,_ = extract_quad_elements(Index)
close_cdb(cdbStat, Index)

# Trouver les éléments traversés
all_crossed_elements = []
for tirant in tirants:
    crossed_elements = find_elements_crossed_by_tirant_optimized(tirant, node_coords, elements)
    all_crossed_elements.extend(crossed_elements)

all_crossed_elements = list(set(all_crossed_elements))  # Éliminer les doublons

# Tracer la structure avec les éléments traversés hachurés en bleu et les tirants verts
plot_structure_with_hachured_elements(file_path, all_crossed_elements, tirants, view_axis='xy')


CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0
CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0
CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0
CDB opened successfully, CDB Status = 3
CDB closed successfully, CDB Status = 0


## IV. Optimiser les tirants